### My notes
There are six action moves wasd, turn lr, attack 
**Couldn't download the doom app from the course, so I downloaded vizdoom**

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import gym
import vizdoom as vzd

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


# import experience_replay, image_preprocessing                                 
# Part 1 - Building the AI

# Making the brain
class CNN(nn.Module): # The input_image is (1x80x80)
    def __init__(self, number_actions, ):
        super(CNN, self).__init__()
        self.convolutional1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=5, stride=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
        )      
        torch.nn.init.xavier_normal_(self.convolutional1.weight)
        # self.convolutional1.apply(init_cnn)
        self.convolutional2 = nn.Sequential(
            nn.Conv2d(32,32,kernel_size=3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
        )         
        torch.nn.init.xavier_normal_(self.convolutional2.weight)

        self.convolutional3 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=2, stride=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3,2)
        )
        torch.nn.init.xavier_normal_(self.convolutional3.weight)

        self.fc1 = nn.Sequential(
            nn.Linear(self.count_neurons((1, 80, 80)), out_features=40),
            nn.Dropout(),
            nn.ReLU()
        )
        torch.nn.init.xavier_normal_(self.convolutional3.weight)
        # self.fc1 = nn.Sequential(nn.Linear(out_features=40), nn.Dropout(), nn.ReLU())
        self.fc2 = nn.Linear(40, number_actions)
        
        # self.apply_init(init_cnn) # make weights xavier_uniform
        
    def count_neurons(self, image_dim): # If we don't use LazyLinear
        x = Variable(torch.rand(1, *image_dim))
        # kernel = 3, stride = 2
        x = self.convolutional1(x)
        x = self.convolutional2(x)
        x = self.convolutional3(x)
        # return x.data.view(1,-1).size(1)
        return x.data.view(1, -1, antialias=True).size(1)
    
    def forward(self, x):
        x = self.convolutional1(x)
        x = self.convolutional2(x)
        x = self.convolutional3(x)
        x = x.view(x.size(0),-1, antialias=True)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
    # def apply_init(self, inputs, init=None):
    #     self.forward(*inputs)
    #     if init is not None:
    #         self.net.apply(init)

# Making the body

# Making the AI

### Now a check that the cnn works
* We see that the accuracy is the accuracy of a LeNet, so it may be better to upgrade the cnn later for better results

In [3]:
import torch.cuda
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torch import nn
import torch.optim as optim
from tqdm.notebook import tqdm


def load_mnist(batch_size=128, resize=(80, 80)):
    # Load the FashionMNIST dataset
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)), transforms.Resize(resize)])
    train_dataset = FashionMNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = FashionMNIST(root='./data', train=False, transform=transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader


def init_cnn(module):
    if type(module) == nn.Linear or type(module) == nn.Conv2d:
        nn.init.xavier_uniform_(module.weight)


def fit(model: nn.Module, train_loader: DataLoader):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    input_data = next(iter(train_loader))[0].to(device)
    # model.apply_init([input_data], init_cnn)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), 0.001)

    total_step = len(train_loader)
    for epoch in range(10):
        epoch_loss = 0.0
        for i, (images, labels) in tqdm(enumerate(train_loader), total=total_step):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Print average epoch loss
        average_loss = epoch_loss / total_step
        print(f"Epoch [{epoch + 1}/10], Average Loss: {average_loss:.4f}")

    return model

In [3]:
train_loader, _ = load_mnist()
model = fit(CNN(10), train_loader)

C:\Users\kosti_0b5rpb8\anaconda3\envs\torch_cuda__118\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [1/10], Average Loss: 0.8105


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [2/10], Average Loss: 0.5635


...

Epoch [8/10], Average Loss: 0.4002


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [9/10], Average Loss: 0.3910


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [10/10], Average Loss: 0.3847


# Now making the body

In [4]:
class SoftmaxBody(nn.Module):
    
    def __init__(self, T):
        super(SoftmaxBody, self).__init__()
        self.T = T
        
    def forward(self, outputs):
        # to get the action, we use softmax
        probs = F.softmax(outputs * self.T)
        actions = probs.multinomial()
        return actions
    

## Now create the AI by combining body and brain 

In [5]:
class AI:
    def __init__(self, brain: CNN, body: SoftmaxBody):
        self.brain = brain
        self.body = body
        
    # The combination of the forward functions
    def __call__(self, inputs): # with __call__ you can treat an AI object as a function
        inp = Variable(torch.from_numpy(np.array(inputs, dtype=np.float32)))
        output = self.brain(inp)
        actions = self.body(output)
        return actions.data.numpy()     # tensor to array
        

# Part 2

Attempting to run vizdoom by the GitHub example

In [6]:
import os
vzd.scenarios_path, os.listdir(vzd.scenarios_path)

('C:\\Users\\kosti_0b5rpb8\\anaconda3\\envs\\torch_cuda__118\\lib\\site-packages\\vizdoom\\scenarios',
 ['basic.cfg',
  'basic.wad',
  'cig.cfg',
  'cig.wad',
  'cig_with_unknown.wad',
  'deadly_corridor.cfg',
  'deadly_corridor.wad',
  'deathmatch.cfg',
  'deathmatch.wad',
  'defend_the_center.cfg',
  'defend_the_center.wad',
  'defend_the_line.cfg',
  'defend_the_line.wad',
  'health_gathering.cfg',
  'health_gathering.wad',
  'health_gathering_supreme.cfg',
  'health_gathering_supreme.wad',
  'learning.cfg',
  'multi.cfg',
  'multi_deathmatch.wad',
  'multi_duel.cfg',
  'multi_duel.wad',
  'my_way_home.cfg',
  'my_way_home.wad',
  'oblige.cfg',
  'predict_position.cfg',
  'predict_position.wad',
  'rocket_basic.cfg',
  'rocket_basic.wad',
  'simpler_basic.cfg',
  'simpler_basic.wad',
  'take_cover.cfg',
  'take_cover.wad'])

In [11]:
# First try to make the environment for doom
# The problem is that the one that the course shows I cannot get, so I have to learn to use vizdoom
import os 
from tqdm import trange
import itertools as it
config_file_path = os.path.join(vzd.scenarios_path, 'simpler_basic.cfg')
# config_file_path = os.path.join(vzd.scenarios_path, "rocket_basic.cfg")
# config_file_path = os.path.join(vzd.scenarios_path, "basic.cfg")
# use GPU if available (don't really know what this does.
import torchvision
torch.backends.cudnn.benchmark = True 
from time import time, sleep
from PIL import Image

resolution = (80, 80)   # in example was (30, 45)
def preprocess(img):
    """Down samples image to resolution"""
    
    img = Image.fromarray(img)
    img = torchvision.transforms.Resize(resolution)(img)
    img = np.array(img, dtype=np.float32)
    # img = img.numpy().astype(np.float32)
    img = np.expand_dims(img, axis=0) # for the batch dim
    return img

def create_simple_game():
    print("Initializing doom...")
    game = vzd.DoomGame()
    game.load_config(config_file_path)
    game.set_window_visible(False) # maybe change this afterward
    game.set_mode(vzd.Mode.PLAYER)
    game.set_screen_format(vzd.ScreenFormat.GRAY8)
    game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
    game.init()
    print("Doom initialized.")
    
    return game

# Training regime
test_episodes_per_epoch = 100
# Other parameters
frame_repeat = 12

model_savefile = "./model-doom.pth"
save_model = True
load_model = False
skip_learning = False

def test(game, agent):
    """Runs a test_episodes_per_epoch episodes and prints the result"""
    print('\nTesting...')
    test_scores = []
    for test_episode in trange(test_episodes_per_epoch, leave=False):
        game.new_episode()
        while not game.is_episode_finished():
            state = preprocess(game.get_state().screen_buffer)
            best_action_index = agent.get_action(state)
            
            game.make_action(actions[best_action_index], frame_repeat)
        r = game.get_total_reward()
        test_scores.append(r)
    
    test_scores = np.array(test_scores)
    
    print(
        "Results: mean: {:.1f} +/- {:.1f},".format(
            test_scores.mean(), test_scores.std()
        ),
        "min: %.1f" % test_scores.min(),
        "max: %.1f" % test_scores.max(),
    )
    

def run(game, agent, actions, num_epochs, frame_repeat, steps_per_epoch=2000):
    """ Run num epochs of training episodes.
     Skip frame_repeat number of frames after each action. """
    start_time = time()
    for epoch in range(num_epochs):
        game.new_episode()
        train_scores = []
        global_step = 0
        print("\nEpoch #" + str(epoch + 1))
        
        for _ in trange(steps_per_epoch, leave=False):
            state = preprocess(game.get_state().screen_buffer)
            action = agent.get_action(state)
            reward = game.make_action(actions[action], frame_repeat)
            done = game.is_episode_finished()
            
            if not done:
                next_state = preprocess(game.get_state().screen_buffer)
            else:
                next_state = np.zeros((1,80,80)).astype(np.float32)     # in the original was (1, 30, 45)
            
            agent.append_memory(state, action, reward, next_state, done)
            
            if global_step > agent.batch_size:
                agent.train()
                
            if done:
                train_scores.append(game.get_total_reward())
                game.new_episode()
            
            global_step += 1
            
        agent.update_target_net()
        train_scores = np.array(train_scores)
        
        print(
            "Results: mean: {:.1f} +/- {:.1f},".format(
                train_scores.mean(), train_scores.std()
            ),
            "min: %.1f," % train_scores.min(),
            "max: %.1f," % train_scores.max(),
        )
        
        test(game, agent)
        if save_model:
            print("Saving the network weights to: ", model_savefile)
            torch.save(agent.q_net, model_savefile)
        print("Total elapsed time: %.2f minutes" % ((time() - start_time) / 60.0))
        
        game.close()
        return agent, game
    
# todo Check the Rainbow algo
class DuelQNet(nn.Module):
    """Single stream with two outputs 
    one for State Values V_π(s) = E[Q_π(s,a)] for a~π(s) -> (LongTerm) 
    one for Advantage A_π(s,a) = Q_π(s,a) - V_π(s)
    The final Q value is:
    Q(s,a; θ,β,γ) = V(s;θ,γ) + Α(s,a;θ,β), where θ,β,γ are the trained weights
    """
    def __init__(self, available_actions_count):
        # todo this guy used stride of 2 and not MaxPooling... Also the guy had 8 instead of 32 and 16 instead of 64
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        torch.nn.init.xavier_normal_(self.conv1[0].weight)
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        torch.nn.init.xavier_normal_(self.conv2[0].weight)

        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        torch.nn.init.xavier_normal_(self.conv3[0].weight)
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        torch.nn.init.xavier_normal_(self.conv4[0].weight)
        
        # self.state_fc = nn.Sequential(nn.Linear(self.count_neurons((1, 80, 80)), out_features=64), nn.ReLU(), nn.Linear(64, 1))
        self.state_fc = nn.Sequential(nn.LazyLinear(64), nn.ReLU(), nn.Linear(64, 1))
        
        # self.advantage_fc = nn.Sequential(nn.Linear(self.count_neurons((1, 80, 80)), 64), nn.ReLU(), nn.Linear(64, available_actions_count))
        self.advantage_fc = nn.Sequential(nn.LazyLinear(64), nn.ReLU(), nn.Linear(64, available_actions_count))
    
    def count_neurons(self, image_dim): # If we don't use LazyLinear
        device = 'cuda'
        x = Variable(torch.rand(1, *image_dim).to(device))
        # kernel = 3, stride = 2
        x = self.conv1(x).to(device)
        x = self.conv2(x).to(device)
        x = self.conv3(x).to(device)
        x = self.conv4(x).to(device)
        # return x.data.view(1,-1).size(1)
        return x.data.view(1, -1).size(1)
    
    def forward(self, x):
        conv_output_size = self.count_neurons((1,80,80))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(-1, conv_output_size)
        x1 = x[:, :(conv_output_size //2)]
        x2 = x[:, (conv_output_size //2):]
        state_value = self.state_fc(x1).reshape(-1,1)
        advantage_values = self.advantage_fc(x2)
        x = (state_value + 
             (advantage_values - advantage_values.mean(dim=1).reshape(-1,1))) # Note that while subtracting the mean in equation helps with identifiability (1-1)
        return x
            
            
# if __name__ == '__main__':
#     game = create_simple_game()
#     n = game.get_available_buttons_size()
#     actions = [list(a) for a in it.product([0, 1], repeat=n)]
#     print(actions)

In [12]:
train_loader, _ = load_mnist()
new_model = fit(DuelQNet(10), train_loader)


C:\Users\kosti_0b5rpb8\anaconda3\envs\torch_cuda__118\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [1/10], Average Loss: 0.3944


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [2/10], Average Loss: 0.2560


...

Epoch [8/10], Average Loss: 0.0818


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [9/10], Average Loss: 0.0740


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch [10/10], Average Loss: 0.0642


Now that we know that the Duel DQN works, we can make the agent

In [13]:
from collections import deque
import random 

class DQNAgent:  # with epsilon (adaptive)-> descending selection policy
    """
    :var target_net: This likely represents a target neural network, which is a copy of the primary Q-network.
         The target network is typically used to stabilize the training process, as it helps prevent the learning
          process from being overly influenced by changes in the Q-network's weights during training.
    :var self.q_net: This is the primary Q-network, which is the network that is actively being trained 
        and updated during the reinforcement learning process
    """
    def __init__(
            self,
            action_size,
            memory_size,
            batch_size,
            discount_factor,
            lr,
            load_model,
            epsilon=1,
            epsilon_decay=0.9996,
            epsilon_min=0.1,
    ):
        self.action_size = action_size
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        self.discount = discount_factor
        self.lr = lr
        self.memory = deque(maxlen=memory_size)
        self.criterion = nn.MSELoss()
        
        if load_model:
            print("Loading model from: ", model_savefile)
            # Single Stream so q_net === target_net
            self.q_net = torch.load(model_savefile)
            self.target_net = torch.load(model_savefile)
            self.epsilon = self.epsilon_min
            
        else:
            print("Initializing new model")
            self.q_net = DuelQNet(action_size).to(device)
            self.target_net = DuelQNet(action_size).to(device)
        
        self.opt = optim.SGD(self.q_net.parameters(), lr=self.lr) # todo maybe use Adam
        
    def get_action(self, state):
        """
        
        :param state: 
        :return: 
        """
        if np.random.uniform() < self.epsilon:
            return random.choice(range(self.action_size))
        else:
            state = np.expand_dims(state, axis=0)   # add batch dim
            state = torch.from_numpy(state).float().to(device)
            action = torch.argmax(self.q_net(state)).item()
            return action
        
    def update_target_net(self): # target net(ground truth) lags a bit to provide stability
        self.target_net.load_state_dict(self.q_net.state_dict())
        
    def append_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def train(self): # what does this train do? is it the play one?
        batch = random.sample(self.memory, self.batch_size)
        batch = np.array(batch, dtype=object)
        
        states = np.stack(batch[:, 0]).astype(float)
        actions = batch[:, 1].astype(int)
        rewards = batch[:, 2].astype(float)
        next_states = np.stack(batch[:, 3]).astype(float)
        dones = batch[:, 4].astype(bool)
        not_dones = ~dones
        
        row_idx = np.arange(self.batch_size)    # used for indexing the batch
        
        # value of the next states with double q learning
        with torch.no_grad():
            next_states = torch.from_numpy(next_states).float().to(device)
            idx = row_idx, np.argmax(self.q_net(next_states).cpu().data.numpy(), 1)
            next_state_values = self.target_net(next_states).cpu().data.numpy()[idx]
            next_state_values = next_state_values[not_dones]
            
        # this defines y = r + discount * max_a q(s', a)
        q_targets = rewards.copy()
        q_targets[not_dones] += self.discount * next_state_values
        q_targets = torch.from_numpy(q_targets).float().to(device)
        
        # this selects only the q values of the actions taken
        idx = row_idx, actions
        states = torch.from_numpy(states).float().to(device)
        action_values = self.q_net(states)[idx].float().to(device)
        
        # todo here in the tutorial would use the Variable thing from torch
        self.opt.zero_grad()  
        td_error = self.criterion(q_targets, action_values)
        td_error.backward()
        self.opt.step()
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        else:
            self.epsilon = self.epsilon_min
            

In [14]:
# Q-learning settings
learning_rate = 0.00025
discount_factor = 0.99
train_epochs = 5
learning_steps_per_epoch = 2000
replay_memory_size = 10000

# NN learning settings
batch_size = 64

episodes_to_watch = 10

if __name__ == '__main__':
    # Initialize game and actions
    game = create_simple_game()
    n = game.get_available_buttons_size()
    actions = [list(a) for a in it.product([0, 1], repeat=n)]
    
    # Initialize our agent with the set parameters
    agent = DQNAgent(
        len(actions),
        lr=learning_rate,
        batch_size=batch_size,
        memory_size = replay_memory_size,
        discount_factor=discount_factor,
        load_model=load_model,
    )
    
    # Run the training for the set number of epochs
    if not skip_learning:
        agent, game = run(
            game,
            agent,
            actions,
            num_epochs=train_epochs,
            frame_repeat=frame_repeat,
            steps_per_epoch=learning_steps_per_epoch,
        )
        
    # Reinitialize the game with a window visible
    game.close()
    game.set_window_visible(True)
    game.set_mode(vzd.Mode.ASYNC_PLAYER)
    game.init()
    
    for _ in range(episodes_to_watch):
        game.new_episode()
        while not game.is_episode_finished():
            state = preprocess(game.get_state().screen_buffer)
            best_action_index = agent.get_action(state)
            
            # Instead of make_action(a, frame_repeat) in order to make the animation smooth
            game.set_action(actions[best_action_index])
            for _ in range(frame_repeat):
                game.advance_action()
                
        # Sleep between episodes
        sleep(1.0)
        score = game.get_total_reward()
        print("Total score: ", score)
    

Initializing doom...
Doom initialized.
Initializing new model

Epoch #1


Results: mean: -19.1 +/- 153.5, min: -380.0, max: 95.0,

Testing...


Results: mean: -0.5 +/- 143.3, min: -355.0 max: 95.0
Saving the network weights to:  ./model-doom.pth
Total elapsed time: 4.15 minutes
Total score:  94.0
Total score:  94.0
Total score:  94.0


ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.

# Now making the one from the Course because I learned how to use gym

In [15]:
import gym
# import VizdoomEnv  # Import the VizdoomEnv module
from vizdoom.gym_wrapper.gym_env_defns import VizdoomScenarioEnv
from image_preprocessing import PreprocessImage

doom_env = VizdoomScenarioEnv(
    scenario_file='deadly_corridor.cfg',  # Specify the path to your scenario file
    frame_skip=4,  # Adjust the frame skip if needed
    max_buttons_pressed=1,  # Adjust the max buttons pressed if needed
    render_mode='human'  # Set the render mode (either 'human' or 'rgb_array')
)

doom_env = PreprocessImage(doom_env, width = 80, height = 80, grayscale = True)
# doom_env = gym.wrappers.ResizeObservation(doom_env, (80, 80))
# doom_env = gym.wrappers.GrayScaleObservation(doom_env)
doom_env = gym.wrappers.RecordVideo(doom_env, "videos/doom.mp4")
number_actions = doom_env.action_space.n

# Building the AI
cnn = CNN(number_actions)
softmax_body = SoftmaxBody(T = 1.0)

ai = AI(brain=cnn, body=softmax_body)



C:\Users\kosti_0b5rpb8\anaconda3\envs\torch_cuda__118\lib\site-packages\vizdoom\gym_wrapper\base_gym_env.py:79: UserWarning: Detected screen format CRCGCB. Only RGB24 and GRAY8 are supported in the Gym wrapper. Forcing RGB24.
  warnings.warn(
C:\Users\kosti_0b5rpb8\anaconda3\envs\torch_cuda__118\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at C:\Users\kosti_0b5rpb8\Desktop\Reinforcement_Learning_python_applications\Doom\videos\doom.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


AttributeError: 'Sequential' object has no attribute 'weight'

In [22]:
a = list(np.linspace(1, 10, 10))
a += [11,12,13]
a

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11, 12, 13]